In [ ]:
# please download original data from https://business.yelp.com/data/resources/open-dataset/

DATASET = 'Yelp'
RAW_PATH = os.path.join('./data/', DATASET)

items = RAW_PATH + "/yelp_academic_dataset_business.json"
reviews = RAW_PATH + "/yelp_academic_dataset_review.json"

# Review data

In [ ]:
import json
review_list = []

with open(reviews, 'r') as file:
    for line in file:
        line = line.strip()
        if line: 
            data_cur = json.loads(line)
            review_list.append(data_cur)

import pandas as pd
review_df = pd.DataFrame(review_list)

review_df['date'] = pd.to_datetime(review_df['date'])

review_df_filtered = review_df[(review_df['date'] >= '2019-01-01') & (review_df['date'] < '2020-01-01')]

In [19]:
def filter_df(df):
    cnt = 0
    while True:
        print(df.shape)
        asin_counts = df['business_id'].value_counts()
        reviewerName_counts = df['user_id'].value_counts()

        asin_to_remove = asin_counts[asin_counts < 5].index
        reviewerName_to_remove = reviewerName_counts[reviewerName_counts < 5].index

        if len(asin_to_remove) == 0 and len(reviewerName_to_remove) == 0:
            break

        df = df[~df['business_id'].isin(asin_to_remove)]
        df = df[~df['user_id'].isin(reviewerName_to_remove)]
        cnt += 1
        # if cnt==2:break
    return df

review_df_filtered_5core = filter_df(review_df_filtered)

(907284, 9)
(301190, 9)
(243098, 9)
(221195, 9)
(214079, 9)
(210638, 9)
(209298, 9)
(208641, 9)
(208314, 9)
(208168, 9)
(208085, 9)
(208051, 9)
(208023, 9)
(208011, 9)
(208007, 9)
(207999, 9)
(207987, 9)
(207980, 9)
(207964, 9)
(207956, 9)
(207952, 9)


# meta file

In [5]:
# -*- coding: utf-8 -*-
"""Convert the Yelp Dataset Challenge dataset from json format to csv.

For more information on the Yelp Dataset Challenge please visit http://yelp.com/dataset_challenge

"""
import argparse
import collections
import csv
import json


def read_and_write_file(json_file_path, csv_file_path, column_names):
    """Read in the json dataset file and write it out to a csv file, given the column names."""
    cnt = 0
    with open(csv_file_path, 'w') as fout:
        csv_file = csv.writer(fout)
        csv_file.writerow(list(column_names))
        with open(json_file_path) as fin:
            for line in fin:
                cnt += 1
                if cnt % 100000 == 0:print(cnt)
                line_contents = json.loads(line)
                csv_file.writerow(get_row(line_contents, column_names))

def get_superset_of_column_names_from_file(json_file_path):
    """Read in the json dataset file and return the superset of column names."""
    column_names = set()
    with open(json_file_path) as fin:
        for line in fin:
            line_contents = json.loads(line)
            column_names.update(
                    set(get_column_names(line_contents).keys())
                    )
    return column_names

def get_column_names(line_contents, parent_key=''):
    """Return a list of flattened key names given a dict.

    Example:

        line_contents = {
            'a': {
                'b': 2,
                'c': 3,
                },
        }

        will return: ['a.b', 'a.c']

    These will be the column names for the eventual csv file.

    """
    column_names = []
    for k, v in line_contents.items():
        column_name = "{0}.{1}".format(parent_key, k) if parent_key else k
        if isinstance(v, collections.abc.MutableMapping):
            
            column_names.extend(
                    get_column_names(v, column_name).items()
                    )
        else:
            column_names.append((column_name, v))
    return dict(column_names)

def get_nested_value(d, key):
    """Return a dictionary item given a dictionary `d` and a flattened key from `get_column_names`.

    Example:
    d = {'a': {'b': {'c': 1}}}
    key = 'a.b.c'
    get_nested_value(d, key) -> 1
    """
    if d is None:
        return None
    if '.' not in key:
        return d.get(key, None)
    base_key, sub_key = key.split('.', 1)
    if base_key not in d:
        return None
    sub_dict = d[base_key]
    return get_nested_value(sub_dict, sub_key)


def get_row(line_contents, column_names):
    """Return a csv compatible row given column names and a dict."""
    row = []
    for column_name in column_names:
        # print(line_contents, column_name)
        line_value = get_nested_value(
                        line_contents,
                        column_name,
                        )
        
        if isinstance(line_value, str):
            row.append('{0}'.format(line_value.encode('utf-8')))
        elif line_value is not None:
            row.append('{0}'.format(line_value))
        else:
            row.append('')
    return row


"""Convert a yelp dataset file from json to csv."""

   
json_file = items
csv_file = '{0}.csv'.format(json_file.split('.json')[0])

column_names = get_superset_of_column_names_from_file(json_file)

read_and_write_file(json_file, csv_file, column_names)

meta_df = pd.read_csv(csv_file)

100000


/var/folders/yh/pffj28r53gb346_s_wmwnns40000gn/T/ipykernel_5256/1082482126.py:114: DtypeWarning: Columns (1,56) have mixed types. Specify dtype option on import or set low_memory=False.
  meta_df = pd.read_csv(csv_file)


In [20]:
# meta_df = pd.read_csv(RAW_PATH+'/meta.csv')
# Only retain items that appear in interaction data
meta_df['business_id'] = meta_df['business_id'].apply(lambda x: x[2:-1])
useful_meta_df = meta_df[meta_df['business_id'].isin(review_df_filtered_5core['business_id'])].reset_index(drop=True)
all_items = set(useful_meta_df['business_id'].values.tolist())

In [35]:
df = useful_meta_df
((df.isnull().sum())/df.shape[0]).sort_values(ascending=False).map(lambda x:"{:.2%}".format(x))

attributes.RestaurantsCounterService     99.92%
attributes.Open24Hours                   99.91%
attributes.DietaryRestrictions           99.89%
attributes.AgesAllowed                   99.82%
attributes.HairSpecializesIn             99.81%
attributes.AcceptsInsurance              99.40%
attributes.BYOBCorkage                   95.74%
attributes.Corkage                       88.26%
attributes.DriveThru                     86.80%
attributes.Smoking                       86.15%
attributes.BYOB                          85.34%
attributes.GoodForDancing                84.01%
attributes.CoatCheck                     82.97%
attributes.BestNights                    80.01%
attributes.Music                         78.50%
attributes.ByAppointmentOnly             78.10%
attributes.BusinessAcceptsBitcoin        76.13%
attributes.WheelchairAccessible          53.02%
attributes.HappyHour                     52.93%
attributes.DogsAllowed                   45.56%
attributes.RestaurantsTableService      

In [21]:
def clean_category(cat):
    if pd.isna(cat):
        return cat  
    if isinstance(cat, float):
        return cat
    if isinstance(cat, bytes):
        return cat.decode('utf-8').strip("b'")
    return cat.strip("b'")
for col in ["attributes.BusinessAcceptsBitcoin",	"attributes.HappyHour",	"attributes.CoatCheck",	"attributes.RestaurantsAttire",	"categories",
           "attributes.RestaurantsReservations",	"attributes.OutdoorSeating",	"attributes.AcceptsInsurance", 
           "attributes.RestaurantsGoodForGroups",	"name",	"attributes.BusinessAcceptsCreditCards",
           "attributes.WiFi",	"attributes.RestaurantsPriceRange2",	"state",	"city",	"attributes.BikeParking",
           "attributes.Caters", "attributes.RestaurantsDelivery",	"attributes.GoodForKids",	"attributes.RestaurantsTakeOut", "attributes.HasTV","attributes.Alcohol",
           ]:
    if col in useful_meta_df.columns:
        # print(col)
        useful_meta_df[col] = useful_meta_df[col].apply(clean_category)

useful_meta_df = useful_meta_df[['business_id', 'name', 'categories', 'city', 'state', 'stars']]

In [45]:
useful_meta_df

,business_id,name,categories,city,state,stars
0,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,"Sushi Bars, Restaurants, Japanese",Philadelphia,PA,4.0
1,ROeacJQwBeh05Rqg7F6TCg,BAP,"Korean, Restaurants",Philadelphia,PA,4.5
2,9OG5YkX1g2GReZM0AskizA,"""Romano's Macaroni Grill""","Restaurants, Italian",Reno,NV,2.5
3,tMkwHmWFUEXrC9ZduonpTg,The Green Pheasant,"Restaurants, Japanese, Seafood",Nashville,TN,4.0
4,QdN72BWoyFypdGJhhI5r7g,Bar One,"Cocktail Bars, Bars, Italian, Nightlife, Resta...",Philadelphia,PA,4.0
...,...,...,...,...,...,...
14582,fWeWzB9STxcX40AgSEQVcw,Arizona-Sonora Desert Museum,"Parks, Zoos, Active Life, Botanical Gardens, M...",Tucson,AZ,4.5
14583,lCo7sEuxlfFeT-nAOR-sbA,Apple Store,"IT Services & Computer Repair, Mobile Phones, ...",Metairie,LA,3.0
14584,2MAQeAqmD8enCT2ZYqUgIQ,The Melting Pot - Nashville,"Fondue, Beer, Wine & Spirits, Food, Restaurants",Nashville,TN,4.0
14585,w_4xUt-1AyY2ZwKtnjW0Xg,Bittercreek Alehouse,"Bars, Gastropubs, Sandwiches, Nightlife, Resta...",Boise,ID,4.5


# build dataset

In [22]:
data_df = review_df_filtered_5core.copy()
out_df = data_df.rename(columns={'business_id': 'item_id', 'user_id': 'user_id', 'date': 'time'})
out_df = out_df[['user_id', 'item_id', 'time']]
out_df = out_df.drop_duplicates(['user_id', 'item_id', 'time'])
out_df = out_df.sort_values(by=['user_id', 'time'], kind='mergesort').reset_index(drop=True)
out_df.head()

,user_id,item_id,time
0,--T_QxqWcEu76n1daMmlLQ,DcBLYSvOuWcNReolRVr12A,2019-03-21 13:17:26
1,--T_QxqWcEu76n1daMmlLQ,_C7QiQQc47AOEv4PE3Kong,2019-03-21 14:49:29
2,--T_QxqWcEu76n1daMmlLQ,oBNrLz4EDhiscSlbOl8uAw,2019-03-21 15:01:36
3,--T_QxqWcEu76n1daMmlLQ,qb28j-FNX1_6xm7u372TZA,2019-03-21 22:48:56
4,--T_QxqWcEu76n1daMmlLQ,kBgiy6ulQRiATWFPZjTPKA,2019-03-21 23:00:40


In [23]:
# reindex (start from 1)
# uids = sorted(out_df['user_id'].unique())
uids = out_df['user_id'].unique()
user2id = dict(zip(uids, range(1, len(uids) + 1)))
# iids = sorted(out_df['item_id'].unique())
iids = out_df['item_id'].unique()
item2id = dict(zip(iids, range(1, len(iids) + 1)))

import json
with open(RAW_PATH+'/item2id.json', 'w') as f:
    json.dump(item2id, f, indent=4)

with open(RAW_PATH+'/user2id.json', 'w') as f:
    json.dump(user2id, f, indent=4)

out_df['user_id'] = out_df['user_id'].apply(lambda x: user2id[x])
out_df['item_id'] = out_df['item_id'].apply(lambda x: item2id[x])
out_df.head()

,user_id,item_id,time
0,1,1,2019-03-21 13:17:26
1,1,2,2019-03-21 14:49:29
2,1,3,2019-03-21 15:01:36
3,1,4,2019-03-21 22:48:56
4,1,5,2019-03-21 23:00:40


In [50]:
useful_meta_df['item_id'] = useful_meta_df['business_id'].apply(lambda x: item2id[x])

# save data
out_df.to_csv(RAW_PATH+'/inter.csv', index=False)
useful_meta_df.to_csv(RAW_PATH+'/meta.csv', index=False)

inter = out_df.drop(columns=['time'])
inter.columns = ['user_id:token', 'item_id:token']
inter.to_csv(RAW_PATH+'/'+DATASET+'.inter', sep='\t', index=False)

df_txt = out_df.drop(columns=['time'])
df_txt.to_csv(RAW_PATH+'/'+ DATASET+'.txt', sep=' ', index=False, header=False)


/var/folders/yh/pffj28r53gb346_s_wmwnns40000gn/T/ipykernel_5256/2251096293.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  useful_meta_df['item_id'] = useful_meta_df['business_id'].apply(lambda x: item2id[x])


# user prompt

In [55]:
# read data
inter = pd.read_csv(RAW_PATH+'/inter.csv')
useful_meta_df = pd.read_csv(RAW_PATH+'/meta.csv')

# seq slide augmentation

def prepare_data_augmentation(df):
    max_item_list_len = 20

    last_uid = None
    uid_list, item_list, target, item_list_length = [], [], [], []

    for index, row in df.iterrows():
        uid, item_id = row['user_id'], row['item_id']
        if last_uid != uid:
            last_uid = uid
            seq = []
        else:
            if len(seq) > max_item_list_len:
                seq = seq[1:]
            uid_list.append(uid)
            item_list.append(seq[:])
            target.append(item_id)
            item_list_length.append(len(seq))
        seq.append(item_id)

    return uid_list, item_list, target, item_list_length
 

uid_list, item_list, target, item_list_length = prepare_data_augmentation(inter)

meta_dict = useful_meta_df.set_index('item_id').T.to_dict()

requests = {}
for i in range(len(item_list)):
    uid, items = uid_list[i], item_list[i]
    # print([uid] + items)
    key = ":".join(map(str, [uid] + items))
    
    meta_list = []
    for item in items:
        meta_text = f"< Business name: {meta_dict[item]['name']}; Categories: {meta_dict[item]['categories']}; Located City: {meta_dict[item]['city']}; Stars: {meta_dict[item]['stars']} >"
        meta_list.append(meta_text)
    if i < 10:
        print(items)
        print(meta_list)
    
    formatted_string = '.\n'.join(meta_list)
    requests[key] = formatted_string

import pickle
with open(RAW_PATH+'/user_prompt_input.pkl', 'wb') as pickle_file:
    pickle.dump(requests, pickle_file)
    
# with open(RAW_PATH+"/user_prompt_input.pkl", 'rb') as pickle_file:
#     question_dic = pickle.load(pickle_file)

system_input = """Assume you are an expert for business recommendation.
You will be provided with the historical order records of a specific user on the Yelp platform, given in the following format:
< Business name: xxx; Categories: xxx; Located City: xxx; Stars: xxx >.\n 
< Business name: xxx; Categories: xxx; Located City: xxx; Stars: xxx >.\n 
....\n 
< Business name: xxx; Categories: xxx; Located City: xxx; Stars: xxx >.\n
Here, each record is enclosed in angle brackets (< >) and contains attribute information separated by semicolons (;). Multiple records are separated by newline characters (\n) and sorted by time.
Business name represents the name of the business. Categories represents the types of services or products the business offers. Located City represents the city where the business is located.
Stars represents the average rating given by users, typically on a scale from 1 to 5.

Requirement: Please summarize the user's specific preferences for businesses. Note that your response should be a coherent paragraph of no more than 100 words.
"""

[1]
['< Business name: "Drago\'s Seafood Restaurant"; Categories: Restaurants, Seafood; Located City: New Orleans; Stars: 3.5 >']
[1, 2]
['< Business name: "Drago\'s Seafood Restaurant"; Categories: Restaurants, Seafood; Located City: New Orleans; Stars: 3.5 >', '< Business name: "Commander\'s Palace"; Categories: French, Restaurants, Cocktail Bars, Nightlife, American (New), Cajun/Creole, Breakfast & Brunch, American (Traditional), Bars; Located City: New Orleans; Stars: 4.5 >']
[1, 2, 3]
['< Business name: "Drago\'s Seafood Restaurant"; Categories: Restaurants, Seafood; Located City: New Orleans; Stars: 3.5 >', '< Business name: "Commander\'s Palace"; Categories: French, Restaurants, Cocktail Bars, Nightlife, American (New), Cajun/Creole, Breakfast & Brunch, American (Traditional), Bars; Located City: New Orleans; Stars: 4.5 >', '< Business name: Ruby Slipper - New Orleans; Categories: Restaurants, American (Traditional), American (New), Cafes, Breakfast & Brunch; Located City: New O

# item prompt

In [62]:
# {'business_id': 'DcBLYSvOuWcNReolRVr12A',
#  'name': '"Drago\'s Seafood Restaurant"',
#  'categories': 'Restaurants, Seafood',
#  'city': 'New Orleans',
#  'state': 'LA',
#  'stars': 3.5}

def item_template(item_dict):
    text = (
        f"The business's name is {item_dict['name']}; "
        f"The business's categories are {item_dict['categories']}; "
        f"The business is located at {item_dict['city']}, {item_dict['state']}; "
        f"The business's average rating is {item_dict['stars']} (the scale is from 1 to 5); "
    )
    return text

item_meta_formatted = {}
for item_id in item2id.values():
    item_meta_formatted[item_id] = item_template(meta_dict[item_id])
    
def complete_prompt_gen(item_info, history=None):
    prompt = f"""
    Assume you are an expert for business recommendation. Please help me analyze a specific business. You are provided with the following information:
    {item_info};

    Requirements:
    1) Please briefly describe the given business.
    2) Please analyze what type of users would enjoy this business, from the aspects of business categories, location, and rating.
    Please provide your answer in JSON format, following this structure:
    {{
    "item summary": "A description of the business, no more than 70 words.", 
    "potential user analysis": "what type of users would enjoy this business from the aspects of business categories, location, and rating, no more than 50 words."
    }}
    """
    return prompt

item_prompt = {}
for item_id in item2id.values():
    item_info = item_meta_formatted[item_id]
    item_prompt[item_id] = complete_prompt_gen(item_info)

import pickle
with open(RAW_PATH+'/item_prompt_input.pkl', 'wb') as pickle_file:
    pickle.dump(item_prompt, pickle_file)